<a href="https://colab.research.google.com/github/joku8/when2meet-minimum-cover/blob/main/when2meet_minimum_cover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

On the when2meet web page, enter this code in the console to download the data. Use the output csv file for the python scipt below.

```JavaScript
function getCSV() {
    result = "Time," + PeopleNames.join(",")+"\n";
    for(let i = 0; i < AvailableAtSlot.length; i++) {
        let slot = $x(`string(//div[@id="GroupTime${TimeOfSlot[i]}"]/@onmouseover)`);
        slot = slot.match(/.*"(.*)".*/)[1];
        result += slot + ",";
        result += PeopleIDs.map(id => AvailableAtSlot[i].includes(id) ? 1 : 0).join(",");
        result+= "\n";
    }
    return result;
}
content = getCSV()

// Create element with <a> tag
const link = document.createElement("a");

// Create a blog object with the file content which you want to add to the file
const file = new Blob([content], { type: 'text/plain' });

// Add file content in the object URL
link.href = URL.createObjectURL(file);

// Add file name
link.download = "when2meet.csv";

// Add click event to <a> tag to save file.
link.click();
URL.revokeObjectURL(link.href);
```



In [ ]:
import pandas as pd

# Takes the downloaded csv from when2meet and generates the sets
# key=student name
# value=set of available times

def generate_name_set(csv_file) :
  # Read the CSV file into a DataFrame
  df = pd.read_csv(csv_file)

  # Drop the "Time" column to focus only on student availability
  student_df = df.drop(columns=["Time"])

  # Dictionary to store the sets for each student
  set_obj = {}

  # Iterate through each student's column
  for name in student_df.columns:
      availability_set = set()

      # Iterate through each row (time block) in the student's column
      for i, value in enumerate(student_df[name]):
          if value == 1:
              availability_set.add(df.loc[i, "Time"])  # Add the corresponding time block

      set_obj[name] = availability_set  # Store the availability set for the student

  # Create a dictionary to store the availability sets for each student
  availability_dict = {}

  # Iterate through each student's availability set
  for name, availability_set in set_obj.items():
      availability_dict[name] = availability_set

  return availability_dict

In [ ]:
import pandas as pd

# Takes the downloaded csv from when2meet and generates the sets
# key=times
# value=students available at that time

def generate_time_set(csv_file) :
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Drop the "Time" column to focus only on student availability
    student_df = df.drop(columns=["Time"])

    # Create a dictionary to store the availability lists for each time
    availability_dict = {}

    # Iterate through each row (time block) in the DataFrame
    for i, row in df.iterrows():
        time = row["Time"]
        availability_list = []

        # Iterate through each column (student) in the row
        for student, value in row.items():
            if student != "Time" and value == 1:
                availability_list.append(student)  # Add the student to the availability list

        availability_dict[time] = availability_list  # Store the availability list for the time

    return availability_dict


In [ ]:
def find_minimum_cover(availability_dict):
    # Convert the availability_dict values to sets for faster intersection operations
    availability_sets = {time: set(students) for time, students in availability_dict.items()}

    all_students = set(student for students in availability_dict.values() for student in students)
    remaining_students = all_students.copy()
    selected_times = []

    while remaining_students:
        best_time = None
        best_cover = set()

        for time, students in availability_sets.items():
            current_cover = students.intersection(remaining_students)

            if not best_cover or len(current_cover) > len(best_cover):
                best_time = time
                best_cover = current_cover

        if best_time is None:
            break

        selected_times.append(best_time)
        remaining_students -= best_cover
        del availability_sets[best_time]

    return selected_times

In [ ]:
availability_set = generate_time_set("target.csv")
min_cover = find_minimum_cover(availability_set)
print('Minimum cover:', min_cover)

minimum cover: ['Friday 02:30:00 PM', 'Monday 09:00:00 AM', 'Monday 11:00:00 AM']
